In [ ]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     |████████████████████████████████| 720 kB 5.0 MB/s 
     |████████████████████████████████| 188 kB 30.1 MB/s 
     |████████████████████████████████| 46 kB 2.1 MB/s 
     |████████████████████████████████| 1.2 MB 22.6 MB/s 
     |████████████████████████████████| 56 kB 2.0 MB/s 
     |████████████████████████████████| 51 kB 166 kB/s 
Mounted at /content/gdrive


In [ ]:
from fastbook import *

# Redes Neuronales Recurrentes para series de tiempo

Vimos que si vamos agregando más y más información del dataset a nuestras redes neuronales convolucionales, lo hacen mejor. 
Es decir, podemos pensar en agregar: 

- Las últimas $k$ mediciones
- El promedio, la mediana, mínimo y máximo
- EWMA con diferentes alfas
- Desviación estándar
- Algunos percentiles (25%, 75%, etc)
- etc

Y si, en principio no sabemos qué será la importante para ayudar a la red
neuronal a hacer la predicción 

Entonces viene la idea increíble: ¿NO sería mejor entrenar una red neuronal para que decidiera qué métricas son las importantes? 
¿O incluso que creara sus propias métricas? 

**Esa es exactamente la idea de las redes neuronales recurrentes**


In [ ]:
import pandas as pd
import torch 
import torch.nn as nn
import torch.nn.functional as F
import fastai.tabular.all as ft
from torch.utils.data import Dataset, DataLoader

In [ ]:
df = pd.read_csv("/content/daily-min-temperatures.csv", index_col='Date', parse_dates=True)
df

,Temp
Date,
1981-01-01,20.7
1981-01-02,17.9
1981-01-03,18.8
1981-01-04,14.6
1981-01-05,15.8
...,...
1990-12-27,14.0
1990-12-28,13.6
1990-12-29,13.5


In [ ]:
T = torch.tensor(df['Temp'], dtype=torch.float32)

In [ ]:
T[:10]

tensor([20.7000, 17.9000, 18.8000, 14.6000, 15.8000, 15.8000, 15.8000, 17.4000, 21.8000, 20.0000])

In [ ]:
T.shape

torch.Size([3650])

In [ ]:
# para escribir un dataset de pytorch, tenemos que sobreescribir dos cosas
# la longitud (__len__) y otro, el corchete (__getitem__)

class TemperaturaDataset(Dataset):
  def __init__(self, T, max_window = 60):
    self.T = T # Es el tensor
    self.max_window = max_window # máxima cantidad de temperaturas que va a haber antes de predecir
    # Por cuestiones prácticas le ponemmos 60 (a mayor número, más lenta)

  def __len__(self):
    return len(self.T) # longitud del dataset = longitud del tensor

  def __getitem__(self, i): 
    d = i-self.max_window # toma los primeros registros desde i-max_window
    first = max(0,d)

    # En caso de que eligas el segundo y no haya 60 registros anteriores
    # se concatena un tensor de menos unos para que no haya problemas
    padding = -torch.ones((max(0, -d),))

    # y lo concatemaos al tensor que le vamos a pasar en la dimension 0
    x = torch.cat((padding, self.T[first:i]), dim=0)
    y = self.T[i] # y va a ser el que se trata de predecir

    return x, y

In [ ]:
# T es mi tensor y max_window la cantidad de temperaturas máximas de las que me voy a apoyar para
# predecir
data_train = TemperaturaDataset(T, max_window=8)

In [ ]:
# Estoy trantando de predecir el primer elemento del tensor, por lo que no tengo información 
# anterior, por eso son puros -1
data_train[0]

(tensor([-1., -1., -1., -1., -1., -1., -1., -1.]), tensor(20.7000))

In [ ]:
# si pongo 
data_train[1]
# Ahora tengo puros -1 excepto uno, y con esa información debo de tratar de predecir el segundo 

(tensor([-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, 20.7000]),
 tensor(17.9000))

# Modelo

In [ ]:
class MySimpleRNN(nn.Module):
  def __init__(self, hidden_size, num_layers):
    super().__init__()

    # Modelo recurrente 
    self.rnn = nn.RNN(input_size=1, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)

    # predicciones
    self.linear = nn.Linear(hidden_size, 1)

  def forward(self, x): 
    # agregamos una capa de unos, esto ([:, :, None]) es por cuestiones de funcionamiento, entre otras cosas
    y, h_n = self.rnn(x[:, :, None]) # y = la predicción; h_n = la hidden, la última capa escondida

    y = y[:, -1, :] # Cambiamos las dimensiones de y
    return self.linear(y).squeeze()

In [ ]:
model = MySimpleRNN(64, 2) # modelo con dos capaz y que hidden_size sea 64

In [ ]:
# Creamos un vector, con batch size de 3
x = torch.randn((3, 10)) # me regresa algo de tamaño tres, porque la batchsize es de tamaño 3
# 10 es el tamaño de la max_window

In [ ]:
model(x)

tensor([-0.1393, -0.1348, -0.1378], grad_fn=<SqueezeBackward0>)

# Manual (con pytorch)

In [ ]:
def rmse_error(yp, y):
  w = (yp-y)
  return torch.sqrt((w*w).mean())

In [ ]:
train_ds = TemperaturaDataset(T[:3200], max_window=40)
valid_ds = TemperaturaDataset(T[3200:], max_window=40)

In [ ]:
train_dl = DataLoader(train_ds, batch_size=256, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=256)

In [ ]:
from torch import optim 
from fastprogress.fastprogress import progress_bar

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
epochs = 20

In [ ]:
pb = progress_bar(range(epochs))
for epoch in pb: 
  for x, y in train_dl: 
    optimizer.zero_grad()
    yp = model(x)
    loss = F.mse_loss(yp, y)
    loss.backward()
    optimizer.step()
    pb.comment = f'loss: {loss.item():.3f}'

Ahora hay que validar...

In [ ]:
model.eval() # Ponemos el modelo en modo evaluación 
rmse_total = 0
for x, y in valid_dl:
  yp = model(x)
  rmse_total += rmse_error(yp, y)

In [ ]:
print(f'RMSE: {rmse_total/len(valid_dl)}')

RMSE: 3.7286369800567627


# Con fastai

In [ ]:
model = MySimpleRNN(64, 2)

In [ ]:
dls = ft.DataLoaders.from_dsets(train_ds, valid_ds, bs=256)

In [ ]:
learn = ft.Learner(dls, model, opt_func=ft.ranger, loss_func=F.smooth_l1_loss, metrics=[F.l1_loss, rmse_error])

In [ ]:
learn.fit_one_cycle(50)

epoch,train_loss,valid_loss,l1_loss,rmse_error,time
0,10.493287,11.217636,11.717636,12.240503,00:00
1,10.443800,11.183337,11.683337,12.207227,00:00
2,10.429450,11.101688,11.601689,12.128018,00:00
3,10.381669,10.942435,11.442435,11.973573,00:00
4,10.267569,10.669504,11.169504,11.709286,00:00
5,10.101878,10.234605,10.734605,11.290054,00:00
6,9.831438,9.566455,10.066076,10.651177,00:00
7,9.426492,8.610843,9.110029,9.748459,00:00
8,8.889101,7.553630,8.051196,8.765111,00:00
9,8.284281,6.723698,7.217591,8.000101,00:01
